In [ ]:
import numpy as np
freq_bands = dict(                #the frequency bands of interest for the analysis
                delta=(1, 4), 
                theta=(4, 8), 
                alpha=(8, 12), 
                beta=(13, 29), 
                gamma=(30, 80))

In [ ]:
log_freq_bands = dict()

In [ ]:
for band in freq_bands:
    lower, upper = freq_bands[band]
    band_range = upper - lower +1
    log_freq_bands[band] = (np.logspace(np.log10(lower), np.log10(upper), band_range))

In [ ]:
log_freq_bands

In [ ]:
import mne
rawfile = "/home/idrael/DATA/MEG/BIDS_clinic/derivatives/sub-BF28011991/meg/BF28011991_Block1_trans_tsss-eve-epo.fif"
epochs = mne.read_epochs(rawfile, preload= True)

In [ ]:
events = epochs.event_id
events

In [ ]:
gr_1 = epochs["Gr_1"].load_data().average

In [ ]:
for event in events.keys():
    print(event)

# Dipole visualization

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from configuration import (subjects, n_jobs, bids_root, use_source_model_for_sourceloc, inv_loose_option,
                            use_fwd_model_for_sourceloc, do_volume_source_loc, source_loc_methods,
                            pick_meg, pick_eeg, concat_raws, signal_to_noise_ratio, minimum_norm_ori,
                            peaks_tmin, peaks_tmax, peaks_mode, peaks_nr_of_points, dip_times)
import mne
from utils.utils import FileNameRetriever, RawPreprocessor, get_peak_points
import glob
from nilearn.plotting import plot_anat

mne.viz.set_3d_backend("pyvista")

fnr =FileNameRetriever(bids_root)
prepper = RawPreprocessor()

snr = signal_to_noise_ratio
lambda2 = 1. / snr ** 2

for subj in subjects:
    meg_folder = fnr.get_filename(subj, "meg")
    epos = glob.glob(meg_folder + "/*epo.fif")
    subjects_dir = fnr.get_filename(subj=subj, file="subjects_dir")
    bem_sol = fnr.get_filename(subj=subj, file=use_fwd_model_for_sourceloc)
    trans_file = fnr.get_single_trans_file(subj)
    for epo in epos:
        epochs = mne.read_epochs(epo)
        print(f"Event-IDs are: {epochs.event_id}")
        events = epochs.event_id
        print(f"\n\n\nevents: {events}")

        noise_cov = mne.compute_covariance(epochs, tmax=-1, 
                                    #projs=, 
                                    method='auto',
                                    n_jobs=n_jobs)

        data_cov = mne.compute_covariance(epochs,
                                    tmin=-0.5, 
                                    tmax=0.3, 
                                    #projs=, 
                                    method='auto',
                                    n_jobs=n_jobs)

        for event in events.keys():
            eventname = str(event)
            if eventname == "ignore_me" or eventname == "AAA":
                pass
            else:
                e = epochs[eventname].load_data().average()
                spike_folder = fnr.get_filename(subj, "spikes")
                e_folder = os.path.join(spike_folder, eventname)
                cp_folder = os.path.join(spike_folder, eventname, "custom_pics")
                cts_folder = os.path.join(spike_folder, eventname, "custom_time_series")
                gp_folder = os.path.join(spike_folder, eventname, "generic_pics")
                folders = [e_folder, cp_folder, cts_folder, gp_folder]
                if not os.path.isdir(e_folder):
                    for f in folders:
                        print(f)
                        os.mkdir(f)
                
                src_file = fnr.get_filename(subj, use_source_model_for_sourceloc)
                if os.path.isfile(src_file):
                    src = mne.read_source_spaces(src_file)
                else:
                    src = mne.setup_source_space(subj, spacing = use_source_model_for_sourceloc, 
                                                subjects_dir = subjects_dir, 
                                                n_jobs=n_jobs, 
                                                verbose=True)

                fwd = mne.make_forward_solution(e.info, src=src, bem=bem_sol,
                                            trans=trans_file, 
                                            meg=pick_meg, eeg=pick_eeg, mindist=0.2, 
                                            ignore_ref=False, 
                                            n_jobs=n_jobs, verbose=True)
                             
                inv = mne.minimum_norm.make_inverse_operator(e.info, forward=fwd, noise_cov=noise_cov, 
                                            loose=inv_loose_option, depth=0.8)
    

            

In [ ]:
for m in source_loc_methods:
    stc_name = "stc_" + m + "_" + eventname
    stc_name = 'stc_' + m
    for start, stop in dip_times.values():
        dip_epoch = e.copy().crop(start, stop).pick('meg')
        ecd = mne.fit_dipole(dip_epoch, noise_cov, bem_sol, trans=trans_file)[0]
        best_idx = np.argmax(ecd.gof)
        best_time = ecd.times[best_idx]
        trans = mne.read_trans(trans_file)
        mri_pos = mne.head_to_mri(ecd.pos, mri_head_t=trans, subject=subj, subjects_dir=subjects_dir)
        t1_file_name = os.path.join(subjects_dir, subj, 'mri', 'T1.mgz')
        stoptime = str(abs(int(stop*1000)))
        if stoptime == "5":
            stoptime = "05"
        title = str(eventname + ' - ECD @ minus ' + stoptime + ' ms')

        # visualization
        
        fig, axes = plt.subplots(nrows=2, ncols=2,
                                    gridspec_kw=dict(width_ratios=[3, 1],
                                    top=0.85))
        fig.suptitle(f"{eventname} - Equivalent current dipole model - minus {stoptime} ms")
        plot_anat(t1_file_name, cut_coords=mri_pos[0], axes=axes[0])
        fig.add_subplot(2,2,1, projection="3d")
        ecd.plot_locations(trans, subj, subjects_dir, mode="orthoview")
        t1_f_name_pic = ('img_ecd_' + eventname + '_' + '_Dipol_' + stoptime + '.png')
        t1_f_name_pic = os.path.join(e_folder, t1_f_name_pic)
        fig.savefig(t1_f_name_pic)
        plt.close("all")

# Hippocampal segmentation

In [ ]:

import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np

In [ ]:
columns = ["Index", "SegId", "NVoxels", "Volume_mm3", "StructName", "normMean", "normStdDev", "normMin", "normMax", "normRange"]
subjects = ["NP21062020", "NP25062020", "NP02072020", "NP16072020"]

In [ ]:
import matplotlib
matplotlib.rcParams.keys()


----
# Report
----

In [ ]:
import numpy as np
from datetime import datetime
import pdb
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from configuration import (subjects, subjects_dir, n_jobs, derivatives_root, extras_dir, freq_bands, session)
from utils.utils import FileNameRetriever
import mne
import glob
from mne import Report
import mne
from mne_bids import read_raw_bids, BIDSPath
import os

mne.viz.set_3d_backend("pyvista")
matplotlib.rcParams["figure.facecolor"] = "black"

fnr = FileNameRetriever(derivatives_root)

def plot_time_course(series, event='GR_1', filename=None):
    fig.suptitle(str(event + ' Time course'), fontsize=12)
    
    ax1 = fig.add_subplot(3, 2, 1)
    ax1.set_title('minus 20 ms')
    ax1.set_xticks([])
    ax1.set_yticks([])
    mpimg_img = mpimg.imread(sorted(series)[0]) 
    ax1.imshow(mpimg_img)
    
    ax2 = fig.add_subplot(3, 2, 2)
    ax2.set_title('minus 15 ms')
    ax2.set_xticks([])
    ax2.set_yticks([])
    mpimg_img = mpimg.imread(sorted(series)[1]) 
    ax2.imshow(mpimg_img)
    
    ax3 = fig.add_subplot(3, 2, 3)
    ax3.set_title('minus 10 ms')
    ax3.set_xticks([])
    ax3.set_yticks([])
    mpimg_img = mpimg.imread(sorted(series)[2]) 
    ax3.imshow(mpimg_img)
    
    ax4 = fig.add_subplot(3, 2, 4)
    ax4.set_title('minus 5 ms')
    ax4.set_xticks([])
    ax4.set_yticks([])
    mpimg_img = mpimg.imread(sorted(series)[3]) 
    ax4.imshow(mpimg_img)
    
    ax5 = fig.add_subplot(3, 2, 5)
    ax5.set_title('peak')
    ax5.set_xticks([])
    ax5.set_yticks([])
    mpimg_img = mpimg.imread(sorted(series)[4]) 
    ax5.imshow(mpimg_img)
    
    ax6 = fig.add_subplot(3, 2, 6)
    ax6.set_title('plus 5 ms')
    ax6.set_xticks([])
    ax6.set_yticks([])
    mpimg_img = mpimg.imread(sorted(series)[5]) 
    ax6.imshow(mpimg_img)
    
    return fig



for subj in subjects:
    subsubj = "sub-" + subj
    meg_folder = os.path.join(derivatives_root, subsubj, "ses-resting", "meg")
    report_folder = fnr.get_filename(subsubj, "report")
    spike_folder = fnr.get_filename(subsubj, "spikes")
    fifs = glob.glob(meg_folder + "/*EvePreproc*.fif")
    for fif in fifs:
        raw = mne.io.read_raw_fif(fif)
        aquisition_date = raw.info['meas_date']
        year = aquisition_date.year
        month = aquisition_date.month
        day = aquisition_date.day
        aquisition_date = (str(day) + '-' + str(month) + '-' + str(year))
        del(raw)
        break
    now = str(datetime.now())

    try:
        title = (subj + ' _MEG_vom_' + aquisition_date + '_Befund-' + now)
        h5title = title = (subj + ' _MEG_vom_' + aquisition_date + '_Befund')
    except NameError as the_error:
        print("Title setting with date failed")
        title = (subj + ' _MEG_Befund-' + now)
        h5title = (subj + ' _MEG_Befund-' + now)

    report = Report(subject=subsubj, subjects_dir=subjects_dir, 
                        title=title, verbose=True, raw_psd=True)


    # Add title image
    cover_file = extras_dir + '/MEG_title.png'
    cover_title = (subj + ' MEG Befund')
    report.add_images_to_section(cover_file, section=cover_title, captions=cover_title) 


    # Epochs
    bids_derivatives = BIDSPath(subject=subj, datatype="meg", session=session, task="resting", root=derivatives_root, processing="tsssTransEvePreproc")    
    raw = read_raw_bids(bids_derivatives)
    events, event_ids = mne.events_from_annotations(raw)
    epochs = mne.Epochs(raw, events=events, event_id=event_ids, tmin=-1.5, tmax=1, baseline=(-1.5,-1), on_missing = "ignore")
    events = epochs.event_id
    print("\n Events are: ", events)
    
    

In [ ]:
if events.keys() != []:
        spike_folder = fnr.get_filename(subsubj, "spikes")
        desired_events = glob.glob(spike_folder + "/*")
        #print (desired_events)
        for e in desired_events:
            e = e.split("/")[-1]
            if e.lower() == "ignore_me" or e.upper() == "AAA" or e.startswith("."):
                print (f"Omitting {e} from Analysis")
            elif e in events:
                print(f"Adding data from {e} to report...")
                # Visualize Topomaps
                cap = str(e) + " --> Topomaps"
                viz_eve = epochs[e].average().crop(-0.2, 0.2)
                times = np.linspace(-0.02, 0.01, 6)
                figs = viz_eve.plot_joint(times=times, show=False)
                
                # Add to report
                for fig in figs:
                    report.add_figs_to_section(fig, captions=cap, section=e)

                # Find generic and custom pics
                generic_pics_folder = os.path.join(spike_folder, e, "generic_pics")
                dSPM_file = glob.glob(generic_pics_folder + "/*_dSPM.png")
                #eLO_file = glob.glob(generic_pics_folder + "/*_eLORETA.png")
                eLO_peak_file = glob.glob(generic_pics_folder + "/*_eLORETA_with_peaks.png")

                custom_pics_folder = os.path.join(spike_folder, e, "custom_pics")
                custom_pics = glob.glob(custom_pics_folder + "/*.png")
                custom_ts_folder = os.path.join(spike_folder, e, "custom_time_series")
                custom_ts = glob.glob(custom_ts_folder + "/*.png")

                # Add to report
                # generics
                caption = e + ' --> dSPM'
                report.add_images_to_section(dSPM_file, captions=caption, section=e)
                caption = str(e) + ' --> eLORETA + peaks'
                report.add_images_to_section(eLO_peak_file, captions=caption, section=e)
                # custom pics
                if custom_pics is not []:
                    for cst in custom_pics:
                        cst_title = cst.split('/')[-1]
                        cst_title = cst_title.split('.')[0]
                        caption = e + ' --> ' + cst_title
                        report.add_images_to_section(cst, section=e, captions=caption)
                if custom_ts is not []:    
                    for cts in custom_ts:
                        caption = e + ' --> Time course'
                        fig = plt.figure(figsize=(30, 30), dpi=150)
                        fig = plot_time_course(sorted(custom_ts), event=e)
                        plt.tight_layout()
                        report.add_figs_to_section(fig, section=e, captions=caption)
                        break


In [ ]:
                # Visualize Topomaps
                cap = str(e) + " --> Topomaps"
                viz_eve = epochs[e].average().crop(-0.2, 0.2)
                times = np.linspace(-0.02, 0.01, 6)
                figs = viz_eve.plot_joint(times=times, show=False)
                
                # Add to report
                for fig in figs:
                    report.add_figs_to_section(fig, captions=cap, section=e)

                # Find generic and custom pics
                generic_pics_folder = os.path.join(spike_folder, e, "generic_pics")
                dSPM_file = glob.glob(generic_pics_folder + "/*_dSPM.png")

                eLO_peak_file = glob.glob(generic_pics_folder + "/*_eLORETA_with_peaks.png")

                custom_pics_folder = os.path.join(spike_folder, e, "custom_pics")
                custom_pics = glob.glob(custom_pics_folder + "/*.png")
                custom_ts_folder = os.path.join(spike_folder, e, "custom_time_series")
                custom_ts = glob.glob(custom_ts_folder + "/*.png")

                # Add to report
                # generics
                caption = e + ' --> dSPM'
                report.add_images_to_section(dSPM_file, captions=caption, section=e)
                caption = str(e) + ' --> eLORETA + peaks'
                report.add_images_to_section(eLO_peak_file, captions=caption, section=e)
                # custom pics
                if custom_pics is not []:
                    for cst in custom_pics:
                        cst_title = cst.split('/')[-1]
                        cst_title = cst_title.split('.')[0]
                        caption = e + ' --> ' + cst_title
                        report.add_images_to_section(cst, section=e, captions=caption)
                if custom_ts is not []:    
                    for cts in custom_ts:
                        caption = e + ' --> Time course'
                        fig = plt.figure(figsize=(30, 30), dpi=150)
                        fig = plot_time_course(sorted(custom_ts), event=e)
                        plt.tight_layout()
                        report.add_figs_to_section(fig, section=e, captions=caption)
                        break